In [1]:
import torch
from torch.profiler import profile, record_function, ProfilerActivity

In [2]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        # self.pool = nn.MaxPool2d(2, 2)
        # self.conv2 = nn.Conv2d(6, 16, 5)
        # self.fc1 = nn.Linear(16 * 5 * 5, 120)
        # self.fc2 = nn.Linear(120, 84)
        # self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # x = self.pool(F.relu(self.conv1(x)))
        x = self.conv1(x)
        # x = self.pool(F.relu(self.conv2(x)))
        # x = torch.flatten(x, 1) # flatten all dimensions except batch
        # x = F.relu(self.fc1(x))
        # x = F.relu(self.fc2(x))
        # x = self.fc3(x)
        return x


net = Net().cuda()

In [3]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [5]:
inputs = torch.randn(4,3,32,32).cuda()
labels = torch.randn(4,6,28,28).cuda()
with profile(activities=[ProfilerActivity.CPU,ProfilerActivity.CUDA],record_shapes=True) as prof:
    optimizer.zero_grad()
    outputs = net(inputs)

    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

prof.export_chrome_trace("cpu03.json")

STAGE:2023-05-09 06:32:17 3757335:3757335 ActivityProfilerController.cpp:311] Completed Stage: Warm Up
STAGE:2023-05-09 06:32:17 3757335:3757335 ActivityProfilerController.cpp:317] Completed Stage: Collection
STAGE:2023-05-09 06:32:17 3757335:3757335 ActivityProfilerController.cpp:321] Completed Stage: Post Processing
